In [1]:
import time
import os

import pandas as pd
import numpy as np
np.set_printoptions(precision=6, suppress=True)

from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error, r2_score

import tensorflow as tf
from tensorflow.keras import *
tf.__version__

'2.3.0'

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

2 Physical GPUs, 2 Logical GPUs


In [3]:
from tensorflow.keras.metrics import Metric
class RSquare(Metric):
    """Compute R^2 score.
     This is also called as coefficient of determination.
     It tells how close are data to the fitted regression line.
     - Highest score can be 1.0 and it indicates that the predictors
       perfectly accounts for variation in the target.
     - Score 0.0 indicates that the predictors do not
       account for variation in the target.
     - It can also be negative if the model is worse.
     Usage:
     ```python
     actuals = tf.constant([1, 4, 3], dtype=tf.float32)
     preds = tf.constant([2, 4, 4], dtype=tf.float32)
     result = tf.keras.metrics.RSquare()
     result.update_state(actuals, preds)
     print('R^2 score is: ', r1.result().numpy()) # 0.57142866
    ```
    """

    def __init__(self, name='r_square', dtype=tf.float32):
        super(RSquare, self).__init__(name=name, dtype=dtype)
        self.squared_sum = self.add_weight("squared_sum", initializer="zeros")
        self.sum = self.add_weight("sum", initializer="zeros")
        self.res = self.add_weight("residual", initializer="zeros")
        self.count = self.add_weight("count", initializer="zeros")

    def update_state(self, y_true, y_pred):
        y_true = tf.convert_to_tensor(y_true, tf.float32)
        y_pred = tf.convert_to_tensor(y_pred, tf.float32)
        self.squared_sum.assign_add(tf.reduce_sum(y_true**2))
        self.sum.assign_add(tf.reduce_sum(y_true))
        self.res.assign_add(
            tf.reduce_sum(tf.square(tf.subtract(y_true, y_pred))))
        self.count.assign_add(tf.cast(tf.shape(y_true)[0], tf.float32))

    def result(self):
        mean = self.sum / self.count
        total = self.squared_sum - 2 * self.sum * mean + self.count * mean**2
        return 1 - (self.res / total)

    def reset_states(self):
        # The state of the metric will be reset at the start of each epoch.
        self.squared_sum.assign(0.0)
        self.sum.assign(0.0)
        self.res.assign(0.0)
        self.count.assign(0.0)

In [4]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
plt.rcParams['figure.figsize'] = ((8/2.54), (6/2.54))
plt.rcParams["font.family"] = "Arial"
plt.rcParams["mathtext.default"] = "rm"
plt.rcParams.update({'font.size': 11})
MARKER_SIZE = 15
cmap_m = ["#f4a6ad", "#f6957e", "#fccfa2", "#8de7be", "#86d6f2", "#24a9e4", "#b586e0", "#d7f293"]
cmap = ["#e94d5b", "#ef4d28", "#f9a54f", "#25b575", "#1bb1e7", "#1477a2", "#a662e5", "#c2f442"]

plt.rcParams['axes.spines.top'] = False
# plt.rcParams['axes.edgecolor'] = 
plt.rcParams['axes.linewidth'] = 1
plt.rcParams['lines.linewidth'] = 1.5
plt.rcParams['xtick.major.width'] = 1
plt.rcParams['xtick.minor.width'] = 1
plt.rcParams['ytick.major.width'] = 1
plt.rcParams['ytick.minor.width'] = 1

# Model test

## hyperparameters

In [6]:
SCREEN_SIZE = 50
DISP_STEPS = 100
TRAINING_EPOCHS = 500
BATCH_SIZE = 32
LEARNING_RATE = 0.001
DIRECTORY = '../data/'

In [7]:
class ConvBlock(layers.Layer):
    def __init__(self, filters, kernel_size, dropout_rate):
        super(ConvBlock, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.dropout_rate = dropout_rate
        
        self.conv1 = layers.Conv2D(self.filters, self.kernel_size,
                                   activation='relu', kernel_initializer='he_normal', padding='same')
        self.batch1 = layers.BatchNormalization()
        self.drop = layers.Dropout(self.dropout_rate)
        self.conv2 = layers.Conv2D(self.filters, self.kernel_size,
                                   activation='relu', kernel_initializer='he_normal', padding='same')
        self.batch2 = layers.BatchNormalization()
        
    def call(self, inp):
        
        inp = self.batch1(self.conv1(inp))
        inp = self.drop(inp)
        inp = self.batch2(self.conv2(inp))
        
        return inp

In [8]:
class DeconvBlock(layers.Layer):
    def __init__(self, filters, kernel_size, strides):
        super(DeconvBlock, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.strides = strides
        
        self.deconv1 = layers.Conv2DTranspose(self.filters, self.kernel_size, strides=self.strides, padding='same')
        
    def call(self, inp):
        
        inp = self.deconv1(inp)
        
        return inp

In [9]:
class UNet(Model):
    def __init__(self):
        super(UNet, self).__init__()
        
        self.conv_block1 = ConvBlock(64, (2, 2), 0.1)
        self.pool1 = layers.MaxPooling2D()
        
        self.conv_block2 = ConvBlock(128, (2, 2), 0.2)
        self.pool2 = layers.MaxPooling2D()
        
        self.conv_block3 = ConvBlock(256, (2, 2), 0.2)
        
        self.deconv_block1 = DeconvBlock(128, (2, 2), (2, 2))
        self.conv_block4 = ConvBlock(128, (2, 2), 0.2)
        
        self.deconv_block2 = DeconvBlock(32, (2, 2), (2, 2))
        self.padding = layers.ZeroPadding2D(((1, 0), (0, 1)))
        self.conv_block5 = ConvBlock(64, (2, 2), 0.1)
        
        self.output_conv = layers.Conv2D(1, (1, 1), activation='sigmoid')
        
    def call(self, inp):
        
        conv1 = self.conv_block1(inp)
        pooled1 = self.pool1(conv1)
        conv2 = self.conv_block2(pooled1)
        pooled2 = self.pool2(conv2)
        
        bottom = self.conv_block3(pooled2)
        
        deconv1 = self.padding(self.deconv_block1(bottom))
        deconv1 = layers.concatenate([deconv1, conv2])
        deconv1 = self.conv_block4(deconv1)
        deconv2 = self.deconv_block2(deconv1)
        deconv2 = layers.concatenate([deconv2, conv1])
        deconv2 = self.conv_block5(deconv2)
        
        return self.output_conv(deconv2)

In [10]:
unet_model = UNet()
opt = tf.optimizers.Adam(learning_rate=LEARNING_RATE)

checkpoint_path = './checkpoints/UNet_50x50_prevnextmask/'
ckpt = tf.train.Checkpoint(unet_model=unet_model, opt=opt)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=10)
if ckpt_manager.checkpoints:
    ckpt.restore(ckpt_manager.checkpoints[-1])
    print ('Checkpoint ' + ckpt_manager.checkpoints[-1][-2:] +' restored.')

Checkpoint 11 restored.


## data loading

In [11]:
l = np.load('./data/tot_dataset_loss_0.30.npz')
MAXS = l['MAXS']
MINS = l['MINS']

In [12]:
TOM_DIRECTORY = '../data/tom/'
file_list = os.listdir(TOM_DIRECTORY)
dataset_list = ['tom/' + file for file in file_list if file.endswith('.csv') and file.startswith('Val')]
dataset_list.sort()
temp = dataset_list

PAP_DIRECTORY = '../data/pap/'
file_list = os.listdir(PAP_DIRECTORY)
dataset_list = ['pap/' + file for file in file_list if file.endswith('.csv') and file.startswith('Val')]
dataset_list.sort()

temp.extend(dataset_list)
dataset_list = temp

In [13]:
def index_selector(x, sr): #x: sorted count indices (argmax), sr: Screen size
    new_indices = []
    while len(x) > 0:
        new_indices.append(x[0])
        x = np.setdiff1d(x, np.arange(x[0] - sr, x[0] + sr), assume_unique=True)
    return np.array(new_indices)

In [14]:
for FILENAME in dataset_list:
    raw_df = pd.read_csv(DIRECTORY + FILENAME, index_col=['MEAS_DATE']).dropna(how='all')
    raw_df = raw_df[~(raw_df == 0).all(axis=1)].interpolate()
    datetime = pd.DatetimeIndex(raw_df.index)
    env_df = raw_df.values
    np.random.seed(3101)
    null_prob = 0.3
    mask = np.random.choice(2, env_df.size, p=[null_prob, 1-null_prob]).reshape(env_df.shape)
    np.random.seed(4564)
    raw_null_prob = 0.3
    raw_indices = np.random.choice(np.arange(int(env_df.shape[0]/48)-1),
                                   replace=False, size=int(env_df.shape[0]/48 * raw_null_prob))
    raw_indices = raw_indices*48
    raw_indices_ext = []
    for elem in raw_indices:
        for _ in range(elem, elem+48):
            raw_indices_ext.append(_)
    raw_indices = np.array(raw_indices_ext)
    raw_indices = np.unique(raw_indices[raw_indices < env_df.shape[0]])
    env_df = (env_df - MINS)/(MAXS - MINS)

    mask[raw_indices, :] = 0
    missing_df = np.ma.array(env_df, mask=1-mask, fill_value=-1)
    missing_df = missing_df.filled()

    recover_df = missing_df.copy()
    recover_df = pd.DataFrame(recover_df, index=raw_df.index, columns=raw_df.columns)
    recover_df.index = pd.DatetimeIndex(recover_df.index)
    filling_indices = np.array([True])
    prev_filling_indices = np.array([False])
    error = 0
    while True:
        new_mask = recover_df.mask(recover_df >= 0, 1).where(recover_df >= 0, 0).astype('int').values
        if np.array_equal(prev_filling_indices, filling_indices):
            error += 1
            if error > 50:
                print('TOO MANY ERRORS!')
                break

        missing_df = recover_df.copy().fillna(-1).values

        temp_current = []
        temp_prev = []
        temp_next = []
        temp_mask = []
        temp_label = []
        missing_counts_c = []
        missing_counts_r = []
        datetimes = []
        for INDEX in range(SCREEN_SIZE*2, missing_df.shape[0]-SCREEN_SIZE):
            _ = np.count_nonzero(missing_df.mean(axis=1)[(INDEX-SCREEN_SIZE*2):INDEX+SCREEN_SIZE] == -1) # number of missing raw
            __ = np.count_nonzero(missing_df[(INDEX-SCREEN_SIZE):INDEX, :] == -1) # number of missing values in current screen
            missing_counts_r.append(_)
            missing_counts_c.append(__)
        missing_counts_r = np.array(missing_counts_r)
        missing_counts_c = np.array(missing_counts_c)
        drop_index = np.arange(0, missing_counts_r.shape[0])
        if error > 1:
            drop_index = drop_index[(missing_counts_c > 0)]
        else:
            drop_index = drop_index[(missing_counts_r < SCREEN_SIZE*2) & (missing_counts_c > 0)]
        count_sort = index_selector(drop_index, SCREEN_SIZE)

        filling_indices = count_sort + SCREEN_SIZE*2
        if len(filling_indices) == 0:
            print('recovery ended.')
            break
        for INDEX_ in filling_indices:
            temp_prev.append(missing_df[(INDEX_-SCREEN_SIZE*2):(INDEX_-SCREEN_SIZE), :])
            temp_current.append(missing_df[(INDEX_-SCREEN_SIZE):INDEX_, :])
            temp_next.append(missing_df[INDEX_:INDEX_+SCREEN_SIZE, :])
            temp_mask.append(new_mask[(INDEX_-SCREEN_SIZE):INDEX_, :])
            datetimes.append(datetime[(INDEX_-SCREEN_SIZE):INDEX_])

        temp_prev = np.stack(temp_prev)
        temp_current = np.stack(temp_current)
        temp_next = np.stack(temp_next)
        temp_mask = np.stack(temp_mask)
        datetimes = np.array(datetimes)

        test_input = np.stack([temp_current, temp_mask], axis = -1)
        test_input = np.concatenate([test_input]*(int(SCREEN_SIZE/env_df.shape[1])), axis=2)

        pred_result = unet_model.predict(test_input[..., :-1])
        avg_pred = []
        OUTLIER = 2
        for __ in range(5):
            temp = []
            for _ in range(int(pred_result.shape[1]/5)):
                temp.append(pred_result[..., _*5:(_+1)*5, 0][..., __])
            temp = np.stack(temp, axis=2)
            temp.sort(axis=2)
            avg_pred.append(temp[..., OUTLIER:-OUTLIER].mean(axis=2))
        avg_pred = np.stack(avg_pred, axis=2)
        masking = test_input[..., -1]
        avg_masking = masking[..., :5]
        final_pred = np.ma.array(avg_pred, mask=avg_masking, fill_value=np.nan).filled()
        pred_df = pd.DataFrame(final_pred.reshape(-1, 5),
                               index = pd.DatetimeIndex(datetimes.reshape(-1, 1)[:, 0]),
                               columns=raw_df.columns)
        recover_df[recover_df == -1] = np.nan
        recover_df = recover_df.combine_first(pred_df)

        prev_filling_indices = filling_indices
    ((MAXS[:5]-MINS[:5])*recover_df + MINS[:5]).to_csv('./results/recovered_%s_UNet_prevnextmask.csv' % (FILENAME.split('/')[-1].split('.')[0]))
    print('%s done.' % FILENAME.split('/')[-1].split('.')[0])

recovery ended.
Val_PF_0001396_tom_env done.
recovery ended.
Val_PF_0002528_tom_env done.
recovery ended.
Val_PF_0002531_tom_env done.
recovery ended.
Val_PF_0002532_tom_env done.
recovery ended.
Val_PF_0001288_pap_env done.
recovery ended.
Val_PF_0001393_pap_env done.
recovery ended.
Val_PF_0001400_pap_env done.
recovery ended.
Val_PF_0002537_pap_env done.


In [15]:
TOM_DIRECTORY = '../data/tom/'
file_list = os.listdir(TOM_DIRECTORY)
dataset_list = ['tom/' + file for file in file_list if file.endswith('.csv') and file.startswith('Val')]
dataset_list.sort()
temp = dataset_list

PAP_DIRECTORY = '../data/pap/'
file_list = os.listdir(PAP_DIRECTORY)
dataset_list = ['pap/' + file for file in file_list if file.endswith('.csv') and file.startswith('Val')]
dataset_list.sort()

temp.extend(dataset_list)
raw_dataset_list = temp

In [16]:
ORI_DIRECTORY = './results/'
file_list = os.listdir(ORI_DIRECTORY)
ori_dataset_list = [file for file in file_list if file.startswith('original') and file.endswith('0.30p_UNet.csv')]
ori_dataset_list.sort()

In [17]:
RES_DIRECTORY = './results/'
file_list = os.listdir(RES_DIRECTORY)
res_dataset_list = [file for file in file_list if file.startswith('recovered') and file.endswith('UNet_prevnext.csv')]
res_dataset_list.sort()

In [18]:
target_labels = {}
target_preds = {}

raw_df = pd.read_csv('../data/pap/PF_0000587_pap_env.csv', index_col='MEAS_DATE')
for original in [_ for _ in ori_dataset_list if '0.30' in _]:
    target_labels['loss_%s' % (original.split('_')[-2])] = {}
    target_preds['loss_%s' % (original.split('_')[-2])] = {}
    for COL in raw_df.columns:
        target_labels['loss_%s' % (original.split('_')[-2])][COL] = []
        target_preds['loss_%s' % (original.split('_')[-2])][COL] = []
for original, recovery in zip([_ for _ in ori_dataset_list if '0.30' in _], [_ for _ in res_dataset_list if '_cur' in _]):
    raw = [_ for _ in raw_dataset_list if original.split('_')[3] in _][0]
    raw_df = pd.read_csv('../data/%s' % raw, index_col='MEAS_DATE')
    ori_df = pd.read_csv('./results/%s' % original, index_col='MEAS_DATE')[51:-51]
    try:
        res_df = pd.read_csv('./results/%s' % recovery, index_col='Unnamed: 0')
    except ValueError:
        res_df = pd.read_csv('./results/%s' % recovery, index_col='MEAS_DATE')
    raw_df.index = pd.DatetimeIndex(raw_df.index)
    ori_df.index = pd.DatetimeIndex(ori_df.index)
    res_df.index = pd.DatetimeIndex(res_df.index)
    
    raw_df = raw_df.where(ori_df.isnull())
    res_df = res_df.mask(raw_df.isnull())
    raw_df = raw_df
    res_df = res_df

    for COL in raw_df.columns:
        if len(res_df[COL].dropna()) == 0:
            continue
        else:
            target_labels['loss_%s' % (original.split('_')[-2])][COL].extend(raw_df[COL].dropna().values)
            target_preds['loss_%s' % (original.split('_')[-2])][COL].extend(res_df[COL].dropna().values)

In [19]:
R2_table = {}
RMSE_table = {}
for LOSS_RATE in [0.3]:
    R2_table['loss_%.2fp' % LOSS_RATE] = []
    RMSE_table['loss_%.2fp' % LOSS_RATE] = []
    
    for COL in raw_df.columns:
        R2_table['loss_%.2fp' % LOSS_RATE].append(r2_score(target_labels['loss_%.2fp' % LOSS_RATE][COL], target_preds['loss_%.2fp' % LOSS_RATE][COL]))
        RMSE_table['loss_%.2fp' % LOSS_RATE].append(np.sqrt(mean_squared_error(target_labels['loss_%.2fp' % LOSS_RATE][COL], target_preds['loss_%.2fp' % LOSS_RATE][COL])))

ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.

In [ ]:
UNet_R2_df = pd.DataFrame(R2_table)
UNet_R2_df[UNet_R2_df < 0] = 0
UNet_RMSE_df = pd.DataFrame(RMSE_table)

In [ ]:
UNet_R2_df